<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/CIFAR/CIFAR_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution Neural Network

In [0]:
import os
import pickle
import urllib
import sys
import tarfile


In [4]:
URL = "https://www.cs.toronto.edu/~kriz/"
filename = "cifar-100-python.tar.gz";
path = os.path.join("datasets", "cifar")
def download_process(count, block_size, total_size):
  percent = count*block_size*100/total_size
  sys.stdout.write("\rDownloading {}%".format(percent))
  sys.stdout.flush()
  
def fetch_pretrain(filename):
  if(os.path.exists(URL+filename)):
    return
  os.makedirs(path, exist_ok=True)
  gz_path = os.path.join(path, filename)
  urllib.request.urlretrieve(URL+filename, gz_path, reporthook=download_process)
  ex_path = tarfile.open(gz_path)
  ex_path.extractall(path=path)
  

fetch_pretrain("cifar-100-python.tar.gz")
  

In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
dict_train = unpickle(os.path.join(path, "cifar-100-python/train"))
dict_test = unpickle(os.path.join(path, "cifar-100-python/test"))

In [0]:
filename_train = dict_train[b'filenames']
fine_label_train = dict_train[b'fine_labels']
data_train = dict_train[b'data']


In [0]:
filename_test = dict_test[b'filenames']
fine_label_test = dict_test[b'fine_labels']
data_test = dict_test[b'data']

In [8]:
data_train[0].shape

(3072,)

In [9]:
import numpy as np
print(data_train.shape)
print(data_test.shape)

(50000, 3072)
(10000, 3072)


In [10]:
X_train = np.concatenate((data_train, data_test))
y_train = np.concatenate((fine_label_train, fine_label_test))
X_train =np.asarray(data_train)
y_train=np.asarray(fine_label_train)



print(X_train.shape)
print(y_train.shape)
print(type(X_train))
print(type(y_train))



(50000, 3072)
(50000,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

URL = "https://www.cs.toronto.edu/~kriz/"
filename = "cifar-100-python.tar.gz";
path = os.path.join("datasets", "cifar")
def download_process(count, block_size, total_size):
  percent = count*block_size*100/total_size
  sys.stdout.write("\rDownloading {}%".format(percent))
  sys.stdout.flush()
  
def fetch_pretrain(filename):
  if(os.path.exists(URL+filename)):
    return
  os.makedirs(path, exist_ok=True)
  gz_path = os.path.join(path, filename)
  urllib.request.urlretrieve(URL+filename, gz_path, reporthook=download_process)
  ex_path = tarfile.open(gz_path)
  ex_path.extractall(path=path)
  

fetch_pretrain("cifar-10-python.tar.gz")
data = []
label = []
for i in range(1, 6):
  batch = unpickle(os.path.join("datasets/cifar/cifar-10-batches-py", "data_batch_"+str(i)))
  
  data.append(batch[b'data'])
  label.append(batch[b'labels'])

for i in range(1, len(data)):
  data[i] = np.concatenate((data[i-1] , data[i]), axis=0)
  label[i]= np.concatenate((label[i-1], label[i]), axis=0)
X_train = data[len(data)-1]
y_train = label[len(label)-1]

print(X_train.shape)
print(y_train.shape)
print(type(X_train))
print(type(y_train))

(50000,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [0]:
import tensorflow as tf

In [0]:
learning_rate = 0.001
num_steps = 2000
batch_size = 128

num_input = 3072
num_classes=100
dropout=0.25

def conv_net(X, n_classes, dropout, reuse, is_training):
  with tf.variable_scope('ConvNet', reuse=reuse):
    x = tf.reshape(X, shape=[-1, 32, 32, 3])
    conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
    conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
    fc1 = tf.contrib.layers.flatten(conv2)
    fc1 =tf.layers.dense(fc1, 1024)
    fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
    out = tf.layers.dense(fc1, n_classes)
  return out

In [0]:
def model_fn(features, labels, mode):
  logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
  logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
  
  pred_classes = tf.argmax(logits_test, axis=1)
  pred_probas = tf.nn.softmax(logits_test)
  if(mode==tf.estimator.ModeKeys.PREDICT):
    return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
  loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
  optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
  train_op = optimizer.minimize(loss_op, global_step = tf.train.get_global_step())
  
  acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
  estim_specs = tf.estimator.EstimatorSpec(mode=mode, predictions=pred_classes, loss=loss_op, train_op=train_op, eval_metric_ops = {'accuracy':acc_op})
  return estim_specs


In [32]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9w5z13l7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f12e00ed208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.float32)

In [33]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x=X_train, y=y_train,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp9w5z13l7/model.ckpt.
INFO:tensorflow:loss = 152.50842, step = 0
INFO:tensorflow:global_step/sec: 80.6285
INFO:tensorflow:loss = 4.607315, step = 100 (1.244 sec)
INFO:tensorflow:global_step/sec: 93.1448
INFO:tensorflow:loss = 4.607752, step = 200 (1.077 sec)
INFO:tensorflow:global_step/sec: 92.742
INFO:tensorflow:loss = 4.6260257, step = 300 (1.078 sec)
INFO:tensorflow:global_step/sec: 94.2559
INFO:tensorflow:loss = 4.548916, step = 400 (1.062 sec)
INFO:tensorflow:global_step/sec: 92.8245
INFO:tensorflow:loss = 4.5421276, step = 500 (1.074 sec)
INFO:tensorflow:global_step/sec: 96.0833
INFO:tensorflow:loss = 4.5496006, step = 600 (1.043 sec)
INFO:tensorflow:global_step/sec: 95.3837
INFO:tensorflow:loss

In [34]:
# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x=X_train, y=y_train,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-06-12:20:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9w5z13l7/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-06-12:20:22
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.1007, global_step = 2000, loss = 4.0331964
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmp9w5z13l7/model.ckpt-2000


{'accuracy': 0.1007, 'global_step': 2000, 'loss': 4.0331964}

# Keras Resnet50

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

import os

nbr_of_clases = 10
validation_percentage = 0.2
resnet_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

def prepare_data_for_resnet50(data_to_transform):
    data = data_to_transform.copy().values
    data = data.reshape(-1, 28, 28) / 255
    data = X_rgb = np.stack([data, data, data], axis=-1)
    return data
y = np.zeros((y_train.shape[0], 10))

for i in range(0, y.shape[0]):
   y[i, int(y_train[0])-1]=1
  
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=validation_percentage)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.applications.resnet50 import ResNet50

model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg'))

model.add(Dropout(0.50))
model.add(Dense(nbr_of_clases, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

94658560/94653016 [==============================] - 9s 0us/step


In [0]:
def fit_model(model, epochs=1, train_test_split=0.0):
    model.fit(X_rgb, y, epochs=epochs, validation_split=train_test_split)
    
def get_fitted_data_generator(data):
    data_generator = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
                                   height_shift_range=0.1, zoom_range=0.1)
    data_generator.fit(data)
    return data_generator
    
def fit_model_generator(model, X_train, y_train, epochs=1, batch=32, validation_data=False, X_val=None, y_val=None):
    image_nbr = np.size(X_train, 0)
    training_data_generator = get_fitted_data_generator(X_train)
    
    if validation_data:
        return model.fit_generator(training_data_generator.flow(X_train, y_train, batch_size=batch), steps_per_epoch=(image_nbr//batch),
                        epochs=epochs, validation_data=(X_val, y_val), verbose=1)
    else:
        return model.fit_generator(training_data_generator.flow(X_train, y_train, batch_size=batch), steps_per_epoch=(image_nbr//batch),
                        epochs=epochs, verbose=1)

In [0]:

model_history = fit_model_generator(model, X_train, y_train, epochs=1, validation_data=True, X_val=X_val, y_val=y_val)

Epoch 1/1
522/522 [==============================] - 162s 311ms/step - loss: 0.0324 - acc: 0.9928 - val_loss: 6.4699e-06 - val_acc: 1.0000


In [0]:
def get_predictions(model, data):
    return np.array([np.argmax(prediction) for prediction in model.predict(data)])

In [0]:
predicted = get_predictions(model, X_val)


In [0]:
yval=[]
for i in range(0, y_val.shape[0]):
  pos=0
  for j in range(0, 9):
    if(y_val[i, j]==1):
      pos=j;
      break
  yval.append(pos)
print(len(yval))

4176


In [0]:
from sklearn.metrics import accuracy_score

print("Validataion : ", accuracy_score(predicted, yval))



Validataion :  1.0


In [0]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 235kB 7.0MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [0]:
path = 
path_file = os.path.join("datasets", "CIFAR")


In [0]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(os.path.join(path, "test.7z")).extractall(path)

PatoolError: ignored

In [17]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
y=mnist.train.labels


In [20]:
print(y[:10])

[7 3 4 6 1 8 1 0 9 8]
